In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import csv

# Параметры
font_path = "times.ttf"  # Путь к шрифту
font_size = 52
chars = "abcdefghijklmnopqrstuvwxyz"
output_folder = "symbols"

# Создание папки для символов
import os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Генерация изображений
font = ImageFont.truetype(font_path, font_size)
for char in chars:
    image = Image.new("L", (font_size, font_size), 255)
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), char, font=font, fill=0)
    image = image.crop(image.getbbox())  # Обрезка белых полей
    image.save(f"{output_folder}/{char}.png")

# Расчет признаков
def calculate_features(image):
    data = np.array(image)
    black_pixels = np.where(data == 0)
    n = len(black_pixels[0])

    # Вес каждой четверти
    h, w = data.shape
    quarters = [
        data[:h//2, :w//2], data[:h//2, w//2:],
        data[h//2:, :w//2], data[h//2:, w//2:]
    ]
    weights = [np.sum(q == 0) for q in quarters]

    # Удельный вес
    area = (h * w) / 4
    specific_weights = [w / area for w in weights]

    # Центр тяжести
    x_c = np.mean(black_pixels[1])
    y_c = np.mean(black_pixels[0])

    # Нормированные координаты
    x_norm = x_c / w
    y_norm = y_c / h

    # Моменты инерции
    I_x = np.sum((black_pixels[0] - y_c) ** 2)
    I_y = np.sum((black_pixels[1] - x_c) ** 2)

    # Нормированные моменты
    I_x_norm = I_x / (h * w)
    I_y_norm = I_y / (h * w)

    # Профили
    x_profile = np.sum(data == 0, axis=0)
    y_profile = np.sum(data == 0, axis=1)

    return {
        "weights": weights,
        "specific_weights": specific_weights,
        "x_c": x_c, "y_c": y_c,
        "x_norm": x_norm, "y_norm": y_norm,
        "I_x": I_x, "I_y": I_y,
        "I_x_norm": I_x_norm, "I_y_norm": I_y_norm,
        "x_profile": x_profile, "y_profile": y_profile
    }

# Сохранение признаков и профилей
with open("features.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["Символ", "Вес_1", "Вес_2", "Вес_3", "Вес_4",
                     "Удельный_вес_1", "Удельный_вес_2", "Удельный_вес_3", "Удельный_вес_4",
                     "X_c", "Y_c", "X_norm", "Y_norm", "I_x", "I_y", "I_x_norm", "I_y_norm"])

    for char in chars:
        image = Image.open(f"{output_folder}/{char}.png").convert("L")
        features = calculate_features(image)

        writer.writerow([char, *features["weights"], *features["specific_weights"],
                         features["x_c"], features["y_c"], features["x_norm"], features["y_norm"],
                         features["I_x"], features["I_y"], features["I_x_norm"], features["I_y_norm"]])

        # Сохранение профилей
        plt.bar(range(len(features["x_profile"])), features["x_profile"])
        plt.title(f"X Profile for {char}")
        plt.savefig(f"{output_folder}/{char}_x_profile.png")
        plt.clf()

        plt.bar(range(len(features["y_profile"])), features["y_profile"])
        plt.title(f"Y Profile for {char}")
        plt.savefig(f"{output_folder}/{char}_y_profile.png")
        plt.clf()

<Figure size 640x480 with 0 Axes>